In [1]:
#XML Library
import xml.etree.ElementTree as ET

In [2]:
#To start I am just using a8.crf to test things out
devFile = "./dev/a8.crf"

In [3]:
#Create XML structure
tree = ET.parse(open(devFile))
root = tree.getroot()

The slides on coreference contain the following:
Typical Coreference Pipeline
-Preprocessor: XML removal, tokenization, sentence and paragraph splitting
-Part of Speech Tagging
-Parsing
-Named Entity Recognizer
-Semantic Class Lookup (usualy via WordNet)
-Candidate NP extraction
-Supervised Learning
-Clustering into Chains

In [4]:
#Part of Speech Tagging to Identify Nouns, Noun Phrases, and Pronouns

In [5]:
#Create a dictionary and the list of each coreference
#I'm not sure which is more practical or if either is even needed
def createCoref():
    global root
    for child in root:
        coref[str(child.get('ID'))] = child.text
        corefs.append(child.text)

In [1]:
#Given text and the coref ID, checks to see if any other coref has the exact same text
#If so, sets found to True and sets REF to the ID of the coreference that it matches
def checkExactMatch(coRefText, coID):
    for cr in coref:
        if coref[cr] == coRefText and str(cr) != coID:
            #print "have a match"
            #print "the coRefText is: " + coRefText + " the coID is: " + coID + " the matched is " + cr
            global REF
            global found 
            found = True
            REF = cr

In [7]:
#Checks to see if the coref has potential for an acronym (actually an initialism) 
#and if that acronym is used
def checkAcronym(coRefText, coID):
    #Check if the coref is all caps
    if coRefText.isupper():
        #check list of corefs to see if any of them could match
        for cr in coref:
            words = str(coref[cr])
            acr = ""
            #check if coref is more than one word
            if len(words.split()) > 1:
                #Get the first letter of each word and build candidate acronym
                for word in words.split():
                    acr += word[0]
                #If candidate acronym matches coref, assign the REF
                if acr == coRefText:
                    global REF
                    global found
                    found = True
                    REF = cr

In [8]:
#Create Coref variables
coref = {}
corefs = []
createCoref()

In [9]:
#Loop through each coref to assign ref
for co in root.iter('COREF'):
    coID = co.attrib['ID']
    coRefText = co.text
    REF = ''
    found = False
    
    # Find the correct REF
    
    #### Check String Match ####
    # check exact match in other corefs
    checkExactMatch(coRefText, coID)
    if found == True:
        co.set('REF',REF)
        continue 
    
    #check acronyms
    checkAcronym(coRefText, coID)
    if found == True:
        co.set('REF',REF)
        continue 
    # check head noun match
    # check substring and partial match
    # check word overlap comparisons
    
    #### Check Lexical Similarity
    # i.e. Ford Motor Company = Ford Co. = Ford
    # i.e. Apple Computer = Apple
    # i.e. Federabl Bureau of Investigation = FBI
    
    #### Look through candidate list of Nouns checking using
    # scoping heuristics and recency
    # number, person, and gender agreement
    # sytactic heuristics
    # semantic compatability
    
    # Set the REF value
    

In [10]:
#Print out the entire XML document
print ET.dump(tree)

<TXT>
 nyt960214.0765 
 A4505 
 BC-PANTEX-FLIGHTS-TEX 
 02-14 
 0535 

BC-<COREF ID="1">PANTEX</COREF>-<COREF ID="2">FLIGHTS</COREF>-TEX
FAA UNDERESTIMATED NUMBER OF FLIGHTS OVER PLUTONIUM STORAGE AREA IN
PANHANDLE
(For use by NYTimes News Service clients)
By HOLLACE WEINER
c.1996 Fort Worth Star-Telegram



The <COREF ID="4">Federal Aviation Administration</COREF> underestimated <COREF ID="5">the number</COREF> of
aircraft flying over the <COREF ID="7">Pantex Weapons Plant</COREF> outside Amarillo,
Texas, where much of the nation's surplus <COREF ID="8" REF="41">plutonium</COREF> is stored,
according to computerized studies under way by the Energy
Department.


The <COREF ID="9" REF="4">FAA</COREF> counted each flight at Amarillo International Airport
once. But military training planes make up to 30 <COREF ID="10">passes</COREF> per <COREF ID="11">flight</COREF>
over <COREF ID="12">the plant</COREF>, according to the Defense Nuclear Facilities Safety
Board.


The secretary of <COREF I

I thought I may use these but as I think things through, these probably won't be used.

In [ ]:
#The defined functions I've looked at so far

#Adds a REF value for the XML node with a specific ID. Probably won't end up using this.
def setReference(ID,REF):
    for co in root.iter('COREF'):
        coID = co.attrib['ID']
        if coID == ID:
            co.set('REF',REF)


        
        

In [ ]:
for cr in coref:
    print coref[cr]
    
for cr in corefs:
    print cr
    
#Prints out the Corefs just to check the values
for child in root:
    print child.attrib, child.text
    
#Saving the full text of the file to fullText
with open(devFile, 'r') as myfile:
    fullText=myfile.read()